# Machine learning and deep learning methods applied to predicting customer status

Purpose: Predict active and not active clients based on the proposed data structure and naive data structure. Here, we implemented Support Vector Machines (SVM), Random Forest (RF), K-nearest Neighbours (KNN) and Lasso.

Author: Gabriel Rodrigues Palma and Rafael de Andrade Moral

# Packages used in the project

In [1]:
# LIME modules
import lime
import lime.lime_tabular

# visualisation modules
import matplotlib.pyplot as plt

# Data manipulation modules
import numpy as np
import pandas as pd

# Machine learning modules
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Deep learning modules
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from keras.layers import LeakyReLU

# Machine learning packages
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import collections
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn import svm
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Lasso
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import LeaveOneOut

# Additional packages
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Conv2DTranspose, UpSampling2D, Flatten, Reshape
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import L1
from keras import metrics
import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf

# Testing GPU from MacOs
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


# Packages used in the project

In [2]:
def create_dataset(path, response_class):
    ''' This function reads and prepare the datasets for applying the ML and DL methods'''
    data = pd.read_csv(path)
    data = data.drop(columns = ['Unnamed: 0', 'Subject'])
    explanatory_variables = data.drop(columns = response_class)
    response_variables = data[response_class]        
    binarizer = LabelBinarizer()
    response_variables = binarizer.fit_transform(response_variables)
    
    onehot_encoder = OneHotEncoder(sparse=False)
    hot_encode_response_variable = onehot_encoder.fit_transform(np.array(response_variables).reshape(-1, 1))          
    
    return(explanatory_variables, 
           hot_encode_response_variable, response_variables)

def check_zero_division_and_get_rates(cm):
    ''' This functions checks for divisions per zeros in the computation of 
       True and false positive rate based on the confusion matrix array. Also, 
       this function returns the checked rates'''
    
    if any(np.sum(cm, axis = 1)==0):            
            tpr = cm[:,1][1]/(np.sum(cm, axis = 1)[1]+1e-16)
            fpr = cm[:,1][0]/(np.sum(cm, axis = 1)[0]+1e-16)    
    else:           
        rates = cm[:,1]/np.sum(cm, axis = 1)    
        fpr = rates[0]    
        tpr = rates[1]
    if np.isnan(tpr):        
        tpr=0
        
    return(tpr, fpr)

def check_and_compute_rates(predictions, 
                            classes, 
                            cm):    
    ''' This function returns the values of true and false positive rate for special cases
       where the division is not possible to obtain automatically based on the confusion
       matrix array provided by sklearn'''
        
    if (sum(predictions) == 0 and sum(classes) == 0):        
        fpr = 0
        tpr = 0
    elif (sum(predictions) == len(predictions) and sum(classes) == len(classes)):        
        fpr = 0
        tpr = 1
    else:            
        tpr, fpr = check_zero_division_and_get_rates(cm)
        
    return(tpr, fpr)
    
def get_rates(y_pred,
              y_true):
    ''' This function get the true and false positive rates based on the 
       predictied'''
            
    cm = confusion_matrix(y_true = y_true, y_pred = y_pred)
    tpr, fpr = check_and_compute_rates(y_pred, y_true, cm)
        
    return(tpr, fpr)

def get_rates_by_cross_validation(raw_data):
    ''' This function obtains the accuracy, true and false positive rates 
       based on the cross k-fold cross validation'''
            
    for train_index, test_index in KFold(n_splits=5, shuffle=True).split(patterns):
        
        x_train, x_test = patterns[train_index], patterns[test_index]
        y_train, y_test = classes[train_index], classes[test_index]
                
        pbp_predictions = pbp_prediction(patterns_array=x_test, 
                                         clustered_patterns = clustered_patterns, 
                                         d_base = d_base, alpha = alpha, 
                   outbreak_p_means = prediction.obtain_p_means_with_distance,
                   outbreak_prediction = prediction.predict_with_distance)
        rates = get_rates(predictions = pbp_predictions, classes = y_test)
        tpr.append(rates[0])
        fpr.append(rates[1])
    
    return(np.mean(tpr[tpr!=np.nan]), np.mean(fpr[fpr!=np.nan]))

def get_statistics(y_pred, y_true):
    ''' This function obtains the following statistics: Accuracy, True positive rate and 
       False positive rate'''

    auc = roc_auc_score(y_score = y_pred,
              y_true = y_true)
    acc = accuracy_score(y_pred = y_pred,
                         y_true = y_true)
    rates = get_rates(y_pred = y_pred,
              y_true = y_true)
    tpr = rates[0]
    fpr = rates[1]
    return([acc, tpr, fpr, auc])

In [40]:
def check_class(lime_importance):
    '''This function classify the class among control or experiment based on the LIME score
    negative scores relates to control and positive experiment'''
    if (lime_importance < 0):
        return('Control')
    else:
        return('Experiment')
def get_lime_importance(explanatory_variables, 
                        response_variables, 
                        trial_name, num_features, 
                        feature_names):
    '''This function obtains the parameters importance based on the LIME method. For that the leave one out cross 
    validation approach was used in order to obtain parameter importance for each subject of the experiment'''
    parameters = []
    loo = LeaveOneOut()
    loo.get_n_splits(explanatory_variables)
    
    
    for train_index, test_index in loo.split(explanatory_variables):
        X_train, X_test = explanatory_variables[train_index], explanatory_variables[test_index]
        y_train, y_test = response_variables[train_index], response_variables[test_index]   
        
        dnn = Sequential()
        dnn.add(Dense(100, input_shape=(num_features,)))
        dnn.add(Dense(150, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(200, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(150, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(46, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(20, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(10, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(2, activation = 'softmax'))
        dnn.compile(optimizer = 'adam', 
                      loss = 'mse', 
                     metrics = ['accuracy', 'TruePositives', 'FalsePositives', 'FalseNegatives', 'TrueNegatives'])
        dnn.fit(X_train, y_train, batch_size = 10, epochs=100, verbose = False)
        dnn_predictions = dnn.predict(X_test)
        
        explainer = lime.lime_tabular.LimeTabularExplainer(X_train, 
                                                           feature_names = feature_names,
                                                           class_names=['Control', 'Experiment'], 
                                                           verbose = False, mode='classification')
        print(np.array(X_test[0]))

        exp = explainer.explain_instance(np.array(X_test[0]), dnn.predict, num_features=num_features)
        lime_results = exp.as_list()
        for feature_index in range(num_features):
            lime_feature_importance = list(lime_results[feature_index])
            lime_feature_importance.append(check_class(list(lime_results[feature_index])[1]))
            lime_feature_importance.append(trial_name)
            parameters.append(lime_feature_importance)
            
    parameters = pd.DataFrame(parameters, columns = ['Features', 'Local \n Importance', 'Class', 'Trial'])
        
    return(parameters)

# Importing Datasets

Before obtaining the performance of the learning algorithms selected for this paper, we need to import the datasets related to the Hidden Markov Models features and the peak features.

## Zscored scale Midline EEG data Trial 12

In [4]:
all_Zscored_scale_explanatory_variables_t1, \
  all_Zscored_scale_hot_encode_response_variable_t1, all_Zscored_scale_response_variable_t1 = create_dataset(path = '../../output_data/New_data/CoordinatesPlusEEGData/ZscoredScaledPlusCoordinateTrial1.csv', 
                                                                                           response_class = 'Class')

In [5]:
all_Zscored_scale_explanatory_variables_t1.head()

,MM_Intercept,MM_Hess,average_peak_magnitude,average_peak_curvature,hmm2_mean.1,hmm2_mean.2,hmm2_sd.1,hmm2_sd.2,hmm2_p1,hmm2_p2,...,hmm5_sd.5,hmm5_p1,hmm5_p2,hmm5_p3,hmm5_p4,hmm5_p5,dis_sum,angle_sum,average_speed,idle_time
0,0.430807,-9.240701,0.899261,-0.002411,0.348,0.216,0.065,0.037,0.515900,0.484100,...,0.012,0.180843,0.240996,0.243103,0.144828,0.190230,0.033748,-1.025618,-0.778047,0.509589
1,0.379726,-28.295271,0.994679,-0.002637,0.209,0.364,0.048,0.092,0.530876,0.469124,...,0.015,0.156503,0.214782,0.282709,0.138557,0.207449,0.253317,-0.076674,-0.078018,-0.583368
2,-0.190530,30.395558,0.096708,-0.002210,0.269,0.147,0.064,0.032,0.404959,0.595041,...,0.009,0.229291,0.181818,0.179896,0.210456,0.198539,0.154093,-0.734181,-0.460115,-0.163000
3,0.018348,11.094229,0.167587,-0.001766,0.264,0.155,0.045,0.034,0.440678,0.559322,...,0.022,0.190485,0.208398,0.214176,0.224576,0.162365,0.158903,0.564604,-0.094401,-0.247073
4,0.738628,-31.642053,1.505026,-0.002939,0.275,0.459,0.053,0.082,0.627565,0.372435,...,0.014,0.237708,0.252226,0.111692,0.255517,0.142857,0.015317,0.747200,-0.304367,0.257368


## Zscored scale Midline EEG data Trial 12

In [6]:
all_Zscored_scale_explanatory_variables_t12, \
  all_Zscored_scale_hot_encode_response_variable_t12, all_Zscored_scale_response_variable_t12 = create_dataset(path = '../../output_data/New_data/CoordinatesPlusEEGData/ZscoredScaledPlusCoordinateTrial12.csv', 
                                                                                           response_class = 'Class')

In [7]:
all_Zscored_scale_explanatory_variables_t12.head()

,MM_Intercept,MM_Hess,average_peak_magnitude,average_peak_curvature,hmm2_mean.1,hmm2_mean.2,hmm2_sd.1,hmm2_sd.2,hmm2_p1,hmm2_p2,...,hmm5_sd.5,hmm5_p1,hmm5_p2,hmm5_p3,hmm5_p4,hmm5_p5,dis_sum,angle_sum,average_speed,idle_time
0,0.505831,3.734294,1.254882,-0.003140,0.394,0.253,0.073,0.048,0.315457,0.684543,...,0.067,0.194532,0.219769,0.256572,0.170347,0.158780,-0.028934,-0.094611,0.049350,-0.523141
1,0.154855,-90.618525,1.082626,-0.002968,0.224,0.442,0.029,0.149,0.590447,0.409553,...,0.153,0.149390,0.257114,0.183943,0.295732,0.113821,0.702254,-0.037036,0.819298,-0.665551
2,-0.381763,-108.664622,-0.051024,-0.001874,0.126,0.261,0.022,0.090,0.470644,0.529356,...,0.091,0.129735,0.188447,0.133523,0.357008,0.191288,-0.607236,-0.624665,-1.338644,0.616143
3,-0.024115,-51.478291,0.385727,-0.002211,0.308,0.171,0.059,0.031,0.423729,0.576271,...,0.048,0.095450,0.329170,0.116860,0.266726,0.191793,-0.005279,3.196325,0.130691,-0.238320
4,0.289524,-99.254789,1.295104,-0.003043,0.418,0.236,0.080,0.059,0.412071,0.587929,...,0.068,0.262227,0.234131,0.123829,0.138398,0.241415,0.744721,-0.773085,0.860736,-0.665551


# Working with Coordinate data

## Obtaining learning algorithms interpretation based on Local Interpretable agnostic Explanation (Lime) methods

Based on the best model select using leave one out cross validation, we are going to check the local interpretation of this algorithm

### Coordinate data Trial 1

In [10]:
selected_columns_m5 = ['MM_Intercept', 'MM_Hess', 'average_peak_magnitude', 
                       'average_peak_curvature', 'hmm5_mean.1',
                       'hmm5_mean.2', 'hmm5_mean.3', 'hmm5_mean.4', 'hmm5_mean.5', 'hmm5_sd.1',
                       'hmm5_sd.2', 'hmm5_sd.3', 'hmm5_sd.4', 'hmm5_sd.5', 'hmm5_p1',
                       'hmm5_p2', 'hmm5_p3', 'hmm5_p4', 'dis_sum', 'angle_sum', 'average_speed', 'idle_time']
selected_columns_m5_array = [elem in selected_columns_m5 for elem in all_Zscored_scale_explanatory_variables_t1.columns]

In [11]:
all_Zscored_scale_explanatory_variables_t1_m5 = all_Zscored_scale_explanatory_variables_t1[selected_columns_m5]

In [25]:
len(all_Zscored_scale_explanatory_variables_t1_m5.columns)

22

In [41]:
lime_results_Trial_1 = get_lime_importance(explanatory_variables = np.array(all_Zscored_scale_explanatory_variables_t1_m5),
                                           response_variables = all_Zscored_scale_hot_encode_response_variable_t1, 
                                           trial_name = 'Trial 1',
                                           num_features = len(selected_columns_m5), 
                                           feature_names = all_Zscored_scale_explanatory_variables_t1_m5.columns.values.tolist())


2023-06-19 21:16:40.532071: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 146ms/step


2023-06-19 21:16:48.283711: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


[ 4.30807440e-01 -9.24070136e+00  8.99260658e-01 -2.41101390e-03
  2.17000000e-01  4.00000000e-01  3.05000000e-01  1.72000000e-01
  2.58000000e-01  1.10000000e-02  5.80000000e-02  1.60000000e-02
  2.40000000e-02  1.20000000e-02  1.80842912e-01  2.40996169e-01
  2.43103448e-01  1.44827586e-01  3.37481092e-02 -1.02561772e+00
 -7.78047038e-01  5.09588943e-01]
157/157 [==============================] - 1s 3ms/step


2023-06-19 21:16:55.610744: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

[ 2.10891966e-01 -5.91812583e+01  8.37202061e-01 -2.60713579e-03
  1.27000000e-01  3.05000000e-01  2.32000000e-01  4.27000000e-01
  1.81000000e-01  2.30000000e-02  2.60000000e-02  1.70000000e-02
  6.00000000e-02  1.30000000e-02  1.94871795e-01  2.14871795e-01
  2.41282051e-01  1.19358974e-01  1.44018315e+00 -5.56367700e-02
 -9.89442842e-01  1.73294556e-01]
157/157 [==============================] - 1s 3ms/step


2023-06-19 21:24:00.845134: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 190ms/step


2023-06-19 21:24:08.962554: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


[-4.85401145e-02 -4.75139830e+00  2.45969996e-01 -2.14568247e-03
  3.07000000e-01  1.79000000e-01  1.09000000e-01  2.55000000e-01
  2.33000000e-01  2.20000000e-02  2.10000000e-02  2.20000000e-02
  7.00000000e-03  9.00000000e-03  1.45390071e-01  2.89361702e-01
  2.87234043e-01  1.29078014e-01 -1.83418107e+00 -1.69421682e+00
  2.66835070e+00 -6.67441410e-01]
157/157 [==============================] - 1s 4ms/step


In [42]:
lime_results_Trial_1.to_csv('../../output_data/New_data/lime_dnn_eegcoordinate_data.csv')

### Coordinate data Trial 12

In [43]:
selected_columns_m5 = ['MM_Intercept', 'MM_Hess', 'average_peak_magnitude', 
                       'average_peak_curvature', 'hmm5_mean.1',
                       'hmm5_mean.2', 'hmm5_mean.3', 'hmm5_mean.4', 'hmm5_mean.5', 'hmm5_sd.1',
                       'hmm5_sd.2', 'hmm5_sd.3', 'hmm5_sd.4', 'hmm5_sd.5', 'hmm5_p1',
                       'hmm5_p2', 'hmm5_p3', 'hmm5_p4', 'dis_sum', 'angle_sum', 'average_speed', 'idle_time']
selected_columns_m5_array = [elem in selected_columns_m5 for elem in all_Zscored_scale_explanatory_variables_t12.columns]

In [44]:
all_Zscored_scale_explanatory_variables_t12_m5 = all_Zscored_scale_explanatory_variables_t12[selected_columns_m5]

In [45]:
lime_results_Trial_12 = get_lime_importance(explanatory_variables = np.array(all_Zscored_scale_explanatory_variables_t12_m5),
                                           response_variables = all_Zscored_scale_hot_encode_response_variable_t12, 
                                           trial_name = 'Trial 1',
                                           num_features = len(selected_columns_m5), 
                                           feature_names = all_Zscored_scale_explanatory_variables_t12_m5.columns.values.tolist())

2023-06-19 21:26:50.321572: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 169ms/step


2023-06-19 21:26:58.546920: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


[ 5.05831463e-01  3.73429373e+00  1.25488205e+00 -3.14013672e-03
  3.39000000e-01  2.97000000e-01  2.00000000e-01  2.61000000e-01
  4.47000000e-01  1.70000000e-02  8.00000000e-03  2.80000000e-02
  1.40000000e-02  6.70000000e-02  1.94532072e-01  2.19768665e-01
  2.56572029e-01  1.70347003e-01 -2.89336702e-02 -9.46109069e-02
  4.93500586e-02 -5.23140701e-01]
157/157 [==============================] - 1s 3ms/step


2023-06-19 21:27:03.760591: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

1/1 [==============================] - 0s 211ms/step


2023-06-19 21:34:51.165906: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


[ 2.13394826e-01 -7.81967379e+00  5.66486970e-01 -2.14662998e-03
  1.45000000e-01  2.93000000e-01  2.14000000e-01  4.07000000e-01
  1.90000000e-01  2.70000000e-02  3.00000000e-02  9.00000000e-03
  5.00000000e-02  7.00000000e-03  2.48958333e-01  1.32291667e-01
  1.68750000e-01  1.36458333e-01 -2.13153643e-01 -3.94138618e-01
 -2.78231920e-01 -2.38319653e-01]
157/157 [==============================] - 1s 3ms/step


2023-06-19 21:34:56.459642: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-06-19 21:35:04.755745: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 371ms/step
[-2.77942166e-01  1.01845428e+01 -2.37984942e-01 -1.80294034e-03
  1.14000000e-01  1.34000000e-01  9.00000000e-02  2.47000000e-01
  1.65000000e-01  3.00000000e-03  7.00000000e-03  1.00000000e-02
  3.40000000e-02  8.00000000e-03  1.65811966e-01  1.36752137e-01
  2.99145299e-01  2.05128205e-01 -1.15902825e+00 -8.14592721e-01
  1.60406590e+00 -5.23140701e-01]
157/157 [==============================] - 1s 4ms/step


In [46]:
lime_results_Trial_12

,Features,Local \n Importance,Class,Trial
0,-0.45 < MM_Hess <= 7.22,0.399624,Experiment,Trial 1
1,-0.63 < idle_time <= -0.24,-0.043337,Control,Trial 1
2,hmm5_sd.3 > 0.02,-0.037397,Control,Trial 1
3,-0.37 < angle_sum <= 0.74,0.032715,Experiment,Trial 1
4,MM_Intercept > 0.20,-0.032688,Control,Trial 1
...,...,...,...,...
853,hmm5_p3 > 0.22,-0.008435,Control,Trial 1
854,0.12 < hmm5_mean.2 <= 0.19,0.007180,Experiment,Trial 1
855,hmm5_mean.1 <= 0.15,-0.006009,Control,Trial 1
856,hmm5_sd.4 > 0.03,0.004631,Experiment,Trial 1


In [47]:
lime_results_Trial_12.to_csv('../../output_data/New_data/lime_dnn_eegcoordinate_data_trial_12.csv')